In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
flights = pd.read_csv('data/flights_clean.csv').sample(10000)
flights

,airline,day,day_of_week,departure_delay,destination_airport,destination_latitude,destination_longitude,distance,month,origin_airport,...,scheduled_arrival,scheduled_departure,scheduled_time,state_destination,state_origin,taxi_in,taxi_out,day_of_year,origin_temperature,destination_temperature
2367867,AA,19,2,18.0,RDU,35.87764,-78.78747,646,5,ORD,...,1289,1105,124.0,NC,IL,4.0,17.0,139,50.4,76.9
938932,WN,21,5,-8.0,LAX,33.94254,-118.40807,451,8,TUS,...,1245,1150,95.0,CA,AZ,9.0,14.0,233,85.8,69.5
644765,AA,5,7,-5.0,PHX,33.43417,-112.00806,1262,7,STL,...,1318,1245,193.0,AZ,MO,3.0,17.0,186,75.9,101.7
3106238,UA,25,5,-3.0,STL,38.74769,-90.35999,1735,9,SFO,...,1006,648,238.0,MO,CA,5.0,21.0,268,66.2,72.7
2190614,MQ,25,6,21.0,CVG,39.04614,-84.66217,812,4,DFW,...,1313,1110,143.0,KY,TX,6.0,22.0,115,68.9,49.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815392,WN,1,6,32.0,MDW,41.78598,-87.75242,745,8,CHS,...,740,670,130.0,IL,SC,7.0,8.0,213,81.2,75.5
2507261,WN,12,5,0.0,MCO,28.42889,-81.31603,973,6,DAL,...,585,370,155.0,FL,TX,6.0,13.0,163,84.0,79.5
197765,UA,27,5,-1.0,HNL,21.31869,-157.92241,2556,3,LAX,...,963,790,353.0,HI,CA,4.0,13.0,86,69.3,75.3
1421233,UA,14,3,-1.0,SAT,29.53369,-98.46978,191,1,IAH,...,1138,1072,66.0,TX,TX,4.0,21.0,14,42.9,40.3


In [4]:
pre_y = flights.departure_delay

In [5]:
pre_X = flights.drop(columns=['departure_delay', 'airline', 'destination_airport',
                              'origin_airport', 'state_destination', 'state_origin'])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(pre_X, pre_y)

In [7]:
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [17]:
# Creating a dataframe that will consist of all combinations of polynomial transformations of the 
# predictors to be considered for interactions

predictor_set = ['day_of_week', 'scheduled_arrival', 'scheduled_departure', 'taxi_out']
from itertools import product
values = np.arange(0,len(predictor_set))
polynomial_transformations = pd.DataFrame(product(values, repeat=len(predictor_set)), columns=predictor_set).loc[1:,]
polynomial_transformations.loc[:,'sum_degree'] = (polynomial_transformations).astype(int).sum(axis=1)
polynomial_transformations.loc[:,'count_zeros'] = (polynomial_transformations == 0).astype(int).sum(axis=1)
polynomial_transformations.sort_values(by = ['count_zeros', 'sum_degree'], ascending=[False, True], inplace=True)
polynomial_transformations.drop(columns = ['count_zeros'], inplace=True)
polynomial_transformations.reset_index(inplace = True, drop = True)

In [18]:
X_train.columns[:5]

Index(['day', 'day_of_week', 'destination_latitude', 'destination_longitude',
       'distance'],
      dtype='object')

In [19]:
#Setting the seed as we are shuffling the data before splitting it into K-folds
np.random.seed(123)
# Shuffling the training set before creating K folds
train = train.sample(frac=1)
k = 5 #5-fold cross validation
fold_size = np.round(train.shape[0]/k)

In [20]:
'departure_delay~'+'+'.join(predictor_set)

'departure_delay~day_of_week+scheduled_arrival+scheduled_departure+taxi_out'

In [24]:
'+'+'+'.join(X_train.columns.difference(predictor_set))

'+day+day_of_year+destination_latitude+destination_longitude+destination_temperature+distance+month+origin_latitude+origin_longitude+origin_temperature+scheduled_time+taxi_in'

In [25]:
# Fill out this function - that is all you need to do to make the code work!

# The function must return the mean k-fold cross validation RMSE for the model
# that has the individual predictors,
# the 'selected_interactions', and the 'interaction_being_tested'

# Uncomment the lines below and fill the function

def KFoldCV(selected_interactions, interaction_being_tested):
    rmses = []
    shuffled = train.sample(frac=1)
    folds = np.array_split(shuffled, k)
    
    for i in range(k):
        fold_train = pd.concat([folds[j] for j in range(k) if j != i])
        fold_test = folds[i]
        model = sm.ols('departure_delay~'+'+'.join(X_train.columns)+'+'+'+'.join(predictor_set)+selected_interactions+interaction_being_tested, data=fold_train).fit()
        rmses.append(mean_squared_error(fold_test.departure_delay, model.predict(fold_test), squared=False))
    
    return sum(rmses)/len(rmses)

In [26]:
# This code implements the algorithm of systematically considering interactions of degree 2 and going upto 
# the interaction of degree 12. For a given degree 'd' the interactions are selected greedily based on 
# highest reduction in the 5-fold cross validation RMSE. Once no more reduction in the 5-fold cross validation
# RMSE is possible using interactions of degree 'd', interaction terms of the next higher degree 'd+1' are considered.

# 5-fold cross validation RMSE of the initial model with the 4 predictors of degree one
cv_previous_model = KFoldCV(selected_interactions = '', interaction_being_tested = '')
interaction_being_tested = '+'
selected_interactions = ''

# Considering interactions of degree 'd' = 2 to 12
for d in np.arange(2,13):
    # Selecting interaction terms of degree = 'd'
    degree_set = polynomial_transformations.loc[polynomial_transformations.sum_degree==d, :]
    
    # Initializing objects to store the interactions of degree 'd' that reduce the
    # 5-fold cross validation RMSEs as compared to the previous model
    interactions_that_reduce_KfoldCV = []; cv_degree = []; 
    
    # Creating another DataFrame that will consist of the updated set of interactions of degree 'd' to be considered
    # as interactions that do not reduce the 5-fold cross validation RMSE will be discarded
    degree_set_updated = pd.DataFrame(columns = degree_set.columns)
    
    # Continue adding interactions of degree 'd' in the model until no interactions reduce 
    # the 5-fold cross-validation RMSE
    while True:
        
        #Iterating over all possible interactions of degree 'd'
        for index, row in degree_set.iterrows():
            
            # Creating the formula expression for the interaction term to be tested
            for predictor in predictor_set:
                interaction_being_tested = interaction_being_tested + ('I('+predictor +'**' +\
                                         str(row[predictor]) + ')*' if row[predictor]>1 else\
                                               predictor + '*' if row[predictor]==1 else '')
            interaction_being_tested = interaction_being_tested[:-1]
            
            # Call the function 'KFoldCV' to find out the 5-fold cross validation error on adding the 
            # interaction term being tested to the model
            cv = KFoldCV(selected_interactions, interaction_being_tested)
            
            # If the interaction term being tested reduces the 5-fold cross validation RMSE as compared to the
            # previous model, then consider adding it to the model
            if cv<cv_previous_model:
                interactions_that_reduce_KfoldCV.append(interaction_being_tested)
                cv_degree.append(cv)
                degree_set_updated = pd.concat([degree_set_updated, row.to_frame().T])
            interaction_being_tested = '+'
        cv_data = pd.DataFrame({'interaction':interactions_that_reduce_KfoldCV, 'cv':cv_degree})
        
        # Sort the interaction terms that reduce the 5-fold cross valdiation RMSE based on their respective
        # 5-fold cross validation RMSE
        cv_data.sort_values(by = 'cv', inplace = True)
        
        # Break the loop if no interaction of degree 'd' reduces the 5-fold cross validation RMSE as
        # compared to the previous model
        if cv_data.shape[0]==0:
            break
            
        # Select the interaction that corresponds to the least 5-fold cross validation RMSE
        selected_interactions = selected_interactions + cv_data.iloc[0,0]
        cv_previous_model = cv_data.iloc[0,1]
        cv_degree = []; interactions_that_reduce_KfoldCV = []
        degree_set = degree_set_updated.copy()
        degree_set_updated = pd.DataFrame(columns = degree_set.columns)
        
        # Print the progress after each model update, i.e., after an interaction term is selected
        print("Degree of interactions being considered:",d, ", 5-fold CV RMSE:", cv_previous_model)

Degree of interactions being considered: 2 , 5-fold CV RMSE: 34.32152710382495
Degree of interactions being considered: 2 , 5-fold CV RMSE: 34.277364534382755
Degree of interactions being considered: 3 , 5-fold CV RMSE: 34.14461604348297
Degree of interactions being considered: 4 , 5-fold CV RMSE: 34.12157413976568


In [27]:
selected_interactions

'+scheduled_departure*taxi_out+day_of_week*scheduled_arrival+scheduled_arrival*I(scheduled_departure**2)+I(scheduled_arrival**3)*scheduled_departure'

In [29]:
model = sm.ols('departure_delay~'+'+'.join(X_train.columns)+'+'+'+'.join(predictor_set)+selected_interactions, data=train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        departure_delay   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     9.621
Date:                Fri, 03 Mar 2023   Prob (F-statistic):           1.96e-32
Time:                        18:16:48   Log-Likelihood:                -37156.
No. Observations:                7500   AIC:                         7.436e+04
Df Residuals:                    7477   BIC:                         7.452e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                       -29.2983     20.985     -1.396      0.163     -70.436      11.839
day                                               0.9080      0.645      1.408      0.159      -0.357       2.173
day_of_week                                       0.1434      0.625      0.230      0.818      -1.082       1.368
destination_latitude                             -0.0645      0.096     -0.673      0.501      -0.253       0.124
destination_longitude                            -0.0519      0.042     -1.225      0.221      -0.135       0.031
distance                                         -0.0020      0.006     -0.327      0.744      -0.014       0.010
month                                            27.8726     19.633      1.420      0.156     -10.615      66.360
origin_latitude                                   0.0125      0.095      0.132      0.895      -0.173       0.198
origin_longitude                                  0.0707      0.050      1.425      0.154      -0.027       0.168
scheduled_arrival                                -0.0012      0.015     -0.078      0.938      -0.031       0.029
scheduled_departure                               0.0079      0.017      0.474      0.636      -0.025       0.041
scheduled_time                                    0.0171      0.049      0.348      0.728      -0.079       0.113
taxi_in                                           0.2480      0.065      3.812      0.000       0.120       0.376
taxi_out                                          0.3545      0.135      2.634      0.008       0.091       0.618
day_of_year                                      -0.9170      0.644     -1.423      0.155      -2.180       0.346
origin_temperature                               -0.0336      0.038     -0.885      0.376      -0.108       0.041
destination_temperature                          -0.0272      0.039     -0.705      0.481      -0.103       0.048
scheduled_departure:taxi_out                     -0.0002      0.000     -1.480      0.139      -0.001    7.46e-05
day_of_week:scheduled_arrival                    -0.0007      0.001     -1.049      0.294      -0.002       0.001
I(scheduled_departure ** 2)                    8.509e-06    1.3e-05      0.653      0.514    -1.7e-05     3.4e-05
scheduled_arrival:I(scheduled_departure ** 2) -7.101e-09    1.2e-08     -0.594      0.553   -3.05e-08    1.63e-08
I(scheduled_arrival ** 3)                      1.493e-08    1.2e-08      1.239      0.215   -8.69e-09    3.86e-08
I(scheduled_arrival ** 3):scheduled_departure -6.406e-12   1.09e-11     -0.586      0.558   -2.78e-11     1.5e-11
==============================================================================
Omnibus:                     7749.494   